# Model

In [ ]:
import glob
import os
import json

### Load weather data from JSON

In [ ]:
list_of_files = glob.glob('data/*')
latest_file = max(list_of_files, key=os.path.getctime)

with open(latest_file, "r") as f:
    data = f.read()
    data = json.loads(data)
    print(data)